UNTUK PENGHITUNGAN FITUR IGNYA JIKA DIBUAT TERPISAH

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

# === 1. Baca Data ===
df = pd.read_excel('data pengujian.xlsx')
fitur_ig = pd.read_excel('10 kata.xlsx')

# Pastikan kolom tidak ada yang kosong
df.dropna(subset=['stem_clean', 'sentiment_label', 'Label Pakar'], inplace=True)

# === 2. Ambil fitur IG (kata-kata penting) ===
top_kata = fitur_ig['Kata'].tolist()

# === 3. Buat Vectorizer BoW berdasar fitur IG ===
vectorizer = CountVectorizer(vocabulary=top_kata)
X = vectorizer.transform(df['stem_clean'])  # hasil: fitur BoW

# === 4. Target: Label Peneliti dan Label Pakar ===
y_peneliti = df['sentiment_label']
y_pakar = df['Label Pakar']

# === 5. Split Data (Train dan Test) ===
X_train, X_test, y_train_p, y_test_p, y_train_k, y_test_k = train_test_split(
    X, y_peneliti, y_pakar, test_size=0.2, random_state=42
)

# === 6. Model Naive Bayes (berdasar label peneliti) ===
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train_p)

# === 7. Prediksi Data Uji ===
y_pred = nb_model.predict(X_test)

# === 8. Evaluasi Perbandingan Label Peneliti vs Pakar ===

# === Tentukan urutan label secara eksplisit ===
ordered_labels = ['positif', 'netral', 'negatif']

# === Evaluasi ===
print("=== Evaluasi Model: Label Peneliti vs Label Pakar ===")
print("Confusion Matrix (urutan: positif, netral, negatif):")
print(confusion_matrix(y_test_k, y_pred, labels=ordered_labels))

print("\nClassification Report:")
print(classification_report(y_test_k, y_pred, labels=ordered_labels, target_names=ordered_labels))

=== Evaluasi Model: Label Peneliti vs Label Pakar ===
Confusion Matrix (urutan: positif, netral, negatif):
[[88  0  0]
 [ 4  0  0]
 [ 7  0  1]]

Classification Report:
              precision    recall  f1-score   support

     positif       0.89      1.00      0.94        88
      netral       0.00      0.00      0.00         4
     negatif       1.00      0.12      0.22         8

    accuracy                           0.89       100
   macro avg       0.63      0.38      0.39       100
weighted avg       0.86      0.89      0.85       100



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [26]:
import pandas as pd

# === 1. Baca data pengujian ===
df = pd.read_excel('data pengujian.xlsx')  # Pastikan kolom: sentiment_label, Label Pakar

# === 2. Bersihkan data yang kosong dan normalisasi ke huruf kecil ===
df = df.dropna(subset=['sentiment_label', 'Label Pakar'])
df['sentiment_label'] = df['sentiment_label'].str.lower()
df['Label Pakar'] = df['Label Pakar'].str.lower()

# === 3. Hitung jumlah sentimen dari masing-masing label ===
count_ai = df['sentiment_label'].value_counts().reindex(['positif', 'negatif', 'netral'], fill_value=0)
count_pakar = df['Label Pakar'].value_counts().reindex(['positif', 'negatif', 'netral'], fill_value=0)

# === 4. Gabungkan ke dalam satu DataFrame untuk perbandingan ===
hasil = pd.DataFrame({
    'Jumlah Sentimen Pakar': count_pakar,
    'Jumlah Sentimen AI': count_ai
})

# === 5. Tampilkan hasil ===
print("\n=== Jumlah Sentimen Positif, Negatif, dan Netral ===")
print(hasil)

# === (Opsional) Simpan ke Excel ===
hasil.to_excel('jumlah_sentimen_pakar_dan_ai.xlsx')



=== Jumlah Sentimen Positif, Negatif, dan Netral ===
         Jumlah Sentimen Pakar  Jumlah Sentimen AI
positif                    425                 407
negatif                     57                  67
netral                      18                  26


CARA LAIN: UNTUK PERHITUNGAN FITUR IG NYA DIJADIKAN SATU DAN LANGSUNG DIPROSES DENGAN PEMODELAN NAIVE BAYESNYA

In [36]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

# === 1. Baca Data ===
df = pd.read_excel('data pengujian.xlsx')
df.dropna(subset=['stem_clean', 'sentiment_label', 'Label Pakar'], inplace=True)

# === 2. Vektorisasi Awal (semua kata) untuk hitung IG ===
vectorizer_all = CountVectorizer()
X_all = vectorizer_all.fit_transform(df['stem_clean'])
y_peneliti = df['sentiment_label']
y_pakar = df['Label Pakar']

# === 3. Hitung Information Gain (IG) ===
ig_scores = mutual_info_classif(X_all, y_peneliti, discrete_features=True)
fitur = vectorizer_all.get_feature_names_out()
df_ig = pd.DataFrame({'Kata': fitur, 'IG': ig_scores})

# Simpan/ambil top-10 fitur terbaik
top_kata = df_ig.sort_values(by='IG', ascending=False).head(10)['Kata'].tolist()

# === 4. Buat Vectorizer ulang berdasarkan fitur IG ===
vectorizer_ig = CountVectorizer(vocabulary=top_kata)
X_selected = vectorizer_ig.transform(df['stem_clean'])  # hasil BoW top IG

# === 5. Split Data (Train dan Test) ===
X_train, X_test, y_train_p, y_test_p, y_train_k, y_test_k = train_test_split(
    X_selected, y_peneliti, y_pakar, test_size=0.2, random_state=42
)

# === 6. Model Naive Bayes (berdasar label peneliti) ===
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train_p)

# === 7. Prediksi Data Uji ===
y_pred = nb_model.predict(X_test)

# === 8. Evaluasi: Bandingkan Label Peneliti (hasil model) vs Label Pakar ===
ordered_labels = ['positif', 'netral', 'negatif']

print("=== Evaluasi Model: Label Peneliti vs Label Pakar ===")
print("Confusion Matrix (urutan: positif, netral, negatif):")
print(confusion_matrix(y_test_k, y_pred, labels=ordered_labels))

print("\nClassification Report:")
print(classification_report(y_test_k, y_pred, labels=ordered_labels, target_names=ordered_labels))


=== Evaluasi Model: Label Peneliti vs Label Pakar ===
Confusion Matrix (urutan: positif, netral, negatif):
[[88  0  0]
 [ 4  0  0]
 [ 7  0  1]]

Classification Report:
              precision    recall  f1-score   support

     positif       0.89      1.00      0.94        88
      netral       0.00      0.00      0.00         4
     negatif       1.00      0.12      0.22         8

    accuracy                           0.89       100
   macro avg       0.63      0.38      0.39       100
weighted avg       0.86      0.89      0.85       100



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
